# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [EXPLORATION](#EXPLORATION)
* [PREPROCESSING](#PREPROCESSING)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

Toronto and New York are the famous places in the world. They are diverse in many ways.We want to explore how much they are similar or dissimilar in aspects from a tourist point of view regarding food, accommodation, beautiful places, and many more.
Both are multicultural as well as the financial hubs of their respective countries. 

## Data <a name="data"></a>

For this problem, we will get the services of Foursquare API to explore the data of two cities, in terms of their neighborhoods. The data also include the information about the places around each neighborhood like restaurants, hotels, coffee shops, parks, theaters, art galleries, museums ... . We selected one Borough from each city to analyze their neighborhoods. Brooklyn from New York and Central Toronto from Toronto. We will use “Clustering” to segment the neighborhoods with similar objects on the basis of each neighborhood data. This will help to locate the tourist’s areas and hubs, and then we can judge the similarity or dissimilarity between two cities on that basis.

## Methodology <a name="methodology"></a>

As we have selected two cities Borough to explore their neighborhoods. The data exploration, analysis and visualization for both boroughs are done in the same way but separately.

## EXPLORATION <a name="EXPLORATION"></a>

For Central Toronto case, we have extracted table of Toronto’s Borough from Wikipedia page. Then we arrange the data according to our requirements. In the arrangement phase, which applied multiple steps including but not limited to, eliminating “Not assigned” values, combine neighborhoods which have same geographical coordinates at each borough and sorted against the concerned borough. For data verification and further exploration, we use Foursquare API to get the coordinates of Central Toronto and explore its neighborhoods. The neighborhoods are further characterized as venues and venue categories.
For Brooklyn, we used a saved data file which is already explored through foursquare API in which we have extracted all the boroughs of New York and then sorted against the concerned borough. Then we explored the Brooklyn neighborhoods as venues and venue categories

## PREPROCESSING <a name="PREPROCESSING"></a>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Segmenting and Clustering Neighborhoods in Toronto

let's just download the data of toronto from wikipedia page and clear it

In [8]:
#downloading Data
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
#Deleting rows with not assigned Borough
df.drop( df[ df['Borough'] == "Not assigned" ].index , inplace=True)
# Geographical Coordinates
df_Geo = pd.read_csv("Geospatial_Coordinates.csv") 
#Merging two Data frame
df=df.merge(df_Geo, on='Postal Code') 
#Selecting Borough "Central Toronto"
Central_Toronto_Data = df.loc[(df['Borough'] == 'Central Toronto')].reset_index(drop=True)
# eliminate 'Postcode' column
Central_Toronto_Data=Central_Toronto_Data.drop(['Postal Code'], axis=1)
Central_Toronto_Data

,Borough,Neighborhood,Latitude,Longitude
0,Central Toronto,Lawrence Park,43.728020,-79.388790
1,Central Toronto,Roselawn,43.711695,-79.416936
2,Central Toronto,Davisville North,43.712751,-79.390197
3,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,Central Toronto,North Toronto West,43.715383,-79.405678
5,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
6,Central Toronto,Davisville,43.704324,-79.388790
7,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
8,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


#### Now we will move towards New York Boroughs. We select "Brooklyn" as a Borough and anylze its neighborhoods later

In [10]:
neighborhoods=pd.read_csv("neighborhoods_NY.csv", index_col=0)
# And make sure that the dataset has all 5 boroughs and 306 neighborhoods.
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

neighborhoods.head()

The dataframe has 5 boroughs and 306 neighborhoods.


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
# Creating new Dataframe Brooklyn_data
Brooklyn_Data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_Data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


Let's get the geographical coordinates of Brooklyn.

In [23]:
address = 'Brooklyn, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


Let's get the geographical coordinates of Central Toronto.

In [22]:
address = 'Central Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_Central_Toronto = location.latitude
longitude_Central_Toronto = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude_Central_Toronto, longitude_Central_Toronto))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


#### Foursquare API

In [15]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'LMXL251DIMB524JCLNKI1H4V5SUC50MCGCIRHWV2OUDVT3DC' # your Foursquare ID
CLIENT_SECRET = 'IWVG4TBUP3E5UYRRFGJ3JTKOQQ1DHVU24C3NH0VMLFV10LMX' # your Foursquare Secret
VERSION = '20180604'
limit = 20
print('Your credentails:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

Your credentails:
CLIENT_ID:LMXL251DIMB524JCLNKI1H4V5SUC50MCGCIRHWV2OUDVT3DC
CLIENT_SECRET:IWVG4TBUP3E5UYRRFGJ3JTKOQQ1DHVU24C3NH0VMLFV10LMX


### VISUALIZATION

##### Before Clustering

#### Central Toronto

In [20]:
# create map of Central Toronto using latitude and longitude values
Map_Central_Toronto = folium.Map(location=[latitude_Central_Toronto,longitude_Central_Toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Central_Toronto_Data['Latitude'], Central_Toronto_Data['Longitude'], Central_Toronto_Data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Map_Central_Toronto)  
    
Map_Central_Toronto

#### Brooklyn

let's visualizar Brooklyn the neighborhoods in it.

In [24]:
# create map of Brooklyn using latitude and longitude values
map_Brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Brooklyn_Data['Latitude'], Brooklyn_Data['Longitude'], Brooklyn_Data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Brooklyn)  
    
map_Brooklyn

## Analysis <a name="analysis"></a>

#### Exploring Neighborhoods in Central Toronto

Let's create a function to get all the nearby venues to all the neighborhoods in Central Toronto

In [29]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
Central_Toronto_venues = getNearbyVenues(names=Central_Toronto_Data['Neighborhood'],
                                   latitudes=Central_Toronto_Data['Latitude'],
                                   longitudes=Central_Toronto_Data['Longitude']
                                  )



Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


Let's check the size of the resulting dataframe

In [30]:
print(Central_Toronto_venues.shape)
Central_Toronto_venues.head()

(112, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
4,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [31]:
Central_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,34,34,34,34,34,34
Davisville North,8,8,8,8,8,8
Forest Hill North & West,4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",1,1,1,1,1,1
North Toronto West,21,21,21,21,21,21
Roselawn,1,1,1,1,1,1
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",16,16,16,16,16,16
"The Annex, North Midtown, Yorkville",24,24,24,24,24,24


Let's find out how many unique categories can be curated from all the returned venues

In [32]:
print('There are {} uniques categories.'.format(len(Central_Toronto_venues['Venue Category'].unique())))

There are 63 uniques categories.


Analyze Each Neighborhood

In [33]:
# one hot encoding
Central_Toronto_onehot = pd.get_dummies(Central_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Central_Toronto_onehot['Neighborhood'] = Central_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Central_Toronto_onehot.columns[-1]] + list(Central_Toronto_onehot.columns[:-1])
Central_Toronto_onehot = Central_Toronto_onehot[fixed_columns]

Central_Toronto_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hotel,Indian Restaurant,Indoor Play Area,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
Central_Toronto_Grouped = Central_Toronto_onehot.groupby('Neighborhood').mean().reset_index()

Let's print each neighborhood along with the top 5 most common venues

In [35]:
Central_Toronto_Grouped = Central_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Central_Toronto_Grouped

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hotel,Indian Restaurant,Indoor Play Area,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.0000,0.0000,0.029412,0.000,0.029412,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000,0.000,0.088235,0.029412,0.000000,0.029412,0.000000,0.029412,0.000,0.0000,0.0,0.029412,0.029412,0.029412,0.000000,0.058824,0.000000,0.000000,0.000,0.029412,0.029412,0.058824,0.00,0.0000,0.000000,0.000000,0.000000,0.029412,0.029412,0.058824,0.000000,0.000000,0.029412,0.000000,0.088235,0.029412,0.000000,0.000000,0.0000,0.0000,0.058824,0.000000,0.029412,0.029412,0.00,0.000000,0.0000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
2,Forest Hill North & West,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.25,0.0000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.250000,0.000000,0.000000,0.000000,0.25,0.000000,0.0000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000
5,North Toronto West,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000,0.000000,0.000000,0.000000,0.047619,0.047619,0.142857,0.095238,0.000000,0.000,0.000,0.047619,0.047619,0.000000,0.000000,0.047619,0.000000,0.000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.047619,0.000000,0.047619,0.00000

In [37]:
num_top_venues = 5

for hood in Central_Toronto_Grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Central_Toronto_Grouped[Central_Toronto_Grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0      Sandwich Place  0.09
1        Dessert Shop  0.09
2         Coffee Shop  0.06
3  Italian Restaurant  0.06
4                Café  0.06


----Davisville North----
               venue  freq
0     Sandwich Place  0.12
1       Dance Studio  0.12
2                Gym  0.12
3               Park  0.12
4  Food & Drink Shop  0.12


----Forest Hill North & West----
                 venue  freq
0                Trail  0.25
1        Jewelry Store  0.25
2     Sushi Restaurant  0.25
3                 Park  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0             Bus Line  0.33
1                 Park  0.33
2          Swim School  0.33
3  American Restaurant  0.00
4                  Pub  0.00


----Moore Park, Summerhill East----
                 venue  freq
0                 Park   1.0
1  American Restaurant   0.0
2           Restaurant   0.0
3    Indian Restaurant   0.0
4     Indoor Play Area   0.0


---

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Central_Toronto_Grouped['Neighborhood']

for ind in np.arange(Central_Toronto_Grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Central_Toronto_Grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Gym,Indian Restaurant,Park
1,Davisville North,Gym,Breakfast Spot,Park,Sandwich Place,Department Store,Food & Drink Shop,Dance Studio,Hotel,Fried Chicken Joint,Garden
2,Forest Hill North & West,Jewelry Store,Park,Trail,Sushi Restaurant,Yoga Studio,Dessert Shop,Diner,Donut Shop,Farmers Market,Flower Shop
3,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Fast Food Restaurant,Dessert Shop,Diner,Donut Shop,Farmers Market,Flower Shop
4,"Moore Park, Summerhill East",Park,Yoga Studio,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint,Food & Drink Shop,Flower Shop


Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [43]:
# set number of clusters
kclusters = 5

Central_Toronto_grouped_clustering = Central_Toronto_Grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Central_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 4, 2, 0, 1, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Central_Toronto_merged = Central_Toronto_Data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Central_Toronto_merged = Central_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Central_Toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Fast Food Restaurant,Dessert Shop,Diner,Donut Shop,Farmers Market,Flower Shop
1,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Yoga Studio,Dance Studio,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint,Food & Drink Shop,Flower Shop,Fast Food Restaurant
2,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Breakfast Spot,Park,Sandwich Place,Department Store,Food & Drink Shop,Dance Studio,Hotel,Fried Chicken Joint,Garden
3,Central Toronto,Forest Hill North & West,43.696948,-79.411307,3,Jewelry Store,Park,Trail,Sushi Restaurant,Yoga Studio,Dessert Shop,Diner,Donut Shop,Farmers Market,Flower Shop
4,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Salon / Barbershop,Café,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant


Finally, let's visualize the resulting clusters

In [45]:
# create map
map_clusters = folium.Map(location=[latitude_Central_Toronto, longitude_Central_Toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Central_Toronto_merged['Latitude'], Central_Toronto_merged['Longitude'], Central_Toronto_merged['Neighborhood'], Central_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

###### Cluster 1(Restaurant and Sandwich Place,Stores,sport places)

In [46]:
Central_Toronto_merged.loc[Central_Toronto_merged['Cluster Labels'] == 0, Central_Toronto_merged.columns[[1] + list(range(5, Central_Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Davisville North,Gym,Breakfast Spot,Park,Sandwich Place,Department Store,Food & Drink Shop,Dance Studio,Hotel,Fried Chicken Joint,Garden
4,North Toronto West,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Salon / Barbershop,Café,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant
5,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,American Restaurant,Grocery Store,History Museum,Donut Shop,Indian Restaurant,Liquor Store,Cosmetics Shop
6,Davisville,Sandwich Place,Dessert Shop,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Gym,Indian Restaurant,Park
8,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,American Restaurant,Sports Bar,Fried Chicken Joint,Vietnamese Restaurant,Light Rail Station,Liquor Store,Restaurant,Pizza Place


##### Cluster 2(Garden,studio,shopes)

In [47]:
Central_Toronto_merged.loc[Central_Toronto_merged['Cluster Labels'] == 1, Central_Toronto_merged.columns[[1] + list(range(5, Central_Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Roselawn,Garden,Yoga Studio,Dance Studio,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint,Food & Drink Shop,Flower Shop,Fast Food Restaurant


##### Cluster 3(Parks,Gym)

In [48]:
Central_Toronto_merged.loc[Central_Toronto_merged['Cluster Labels'] == 2, Central_Toronto_merged.columns[[1] + list(range(5, Central_Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"Moore Park, Summerhill East",Park,Yoga Studio,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint,Food & Drink Shop,Flower Shop


##### Cluster 4(Forest,Store,shopes)

In [50]:
Central_Toronto_merged.loc[Central_Toronto_merged['Cluster Labels'] == 3, Central_Toronto_merged.columns[[1] + list(range(5, Central_Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Forest Hill North & West,Jewelry Store,Park,Trail,Sushi Restaurant,Yoga Studio,Dessert Shop,Diner,Donut Shop,Farmers Market,Flower Shop


##### Cluster5(Park,Shcool,Shopes)

In [51]:
Central_Toronto_merged.loc[Central_Toronto_merged['Cluster Labels'] == 4, Central_Toronto_merged.columns[[1] + list(range(5, Central_Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Fast Food Restaurant,Dessert Shop,Diner,Donut Shop,Farmers Market,Flower Shop


#### Exploring Neighborhoods in Brooklyn

In [54]:
Brooklyn_venues = getNearbyVenues(names=Brooklyn_Data['Neighborhood'],
                                   latitudes=Brooklyn_Data['Latitude'],
                                   longitudes=Brooklyn_Data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [55]:
print(Brooklyn_venues.shape)
Brooklyn_venues.head()

(2719, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,The Bookmark Shoppe,40.624577,-74.030562,Bookstore


Let's check how many venues were returned for each neighborhood

In [56]:
Brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,47,47,47,47,47,47
Bay Ridge,87,87,87,87,87,87
Bedford Stuyvesant,29,29,29,29,29,29
Bensonhurst,31,31,31,31,31,31
Bergen Beach,8,8,8,8,8,8
Boerum Hill,93,93,93,93,93,93
Borough Park,19,19,19,19,19,19
Brighton Beach,44,44,44,44,44,44
Broadway Junction,18,18,18,18,18,18


Let's find out how many unique categories can be curated from all the returned venues

In [57]:
print('There are {} uniques categories.'.format(len(Brooklyn_venues['Venue Category'].unique())))

There are 284 uniques categories.


Analyze Each Neighborhood

In [58]:
# one hot encoding
Brooklyn_onehot = pd.get_dummies(Brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Brooklyn_onehot['Neighborhood'] = Brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]

Brooklyn_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,

In [59]:
Brooklyn_onehot.shape

(2719, 284)

Let's print each neighborhood along with the top 5 most common venues

In [60]:
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
Brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,

In [61]:
num_top_venues = 5

for hood in Brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Brooklyn_grouped[Brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0    Chinese Restaurant  0.06
1              Pharmacy  0.06
2      Sushi Restaurant  0.04
3  Fast Food Restaurant  0.04
4           Pizza Place  0.04


----Bay Ridge----
                venue  freq
0                 Spa  0.07
1  Italian Restaurant  0.07
2         Pizza Place  0.06
3    Greek Restaurant  0.05
4                 Bar  0.03


----Bedford Stuyvesant----
           venue  freq
0            Bar  0.07
1    Pizza Place  0.07
2    Coffee Shop  0.07
3           Café  0.07
4  Deli / Bodega  0.07


----Bensonhurst----
                venue  freq
0       Grocery Store  0.06
1  Italian Restaurant  0.06
2          Donut Shop  0.06
3    Sushi Restaurant  0.06
4  Chinese Restaurant  0.06


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.25
1          Donut Shop  0.12
2      Baseball Field  0.12
3   Other Repair Shop  0.12
4  Athletics & Sports  0.12


----Boerum Hill----
                    venue  freq
0          

4                 Park  0.05


----North Side----
                 venue  freq
0          Coffee Shop  0.09
1  American Restaurant  0.05
2          Pizza Place  0.05
3          Yoga Studio  0.04
4             Wine Bar  0.04


----Ocean Hill----
                             venue  freq
0                    Deli / Bodega  0.16
1  Southern / Soul Food Restaurant  0.08
2              Fried Chicken Joint  0.08
3                       Playground  0.08
4                Convenience Store  0.04


----Ocean Parkway----
                  venue  freq
0         Grocery Store  0.08
1     Indian Restaurant  0.04
2  Gym / Fitness Center  0.04
3      Sushi Restaurant  0.04
4             Gift Shop  0.04


----Paerdegat Basin----
              venue  freq
0     Moving Target   0.2
1   Harbor / Marina   0.2
2              Food   0.2
3               Gym   0.2
4  Asian Restaurant   0.2


----Park Slope----
                   venue  freq
0            Coffee Shop  0.10
1           Burger Joint  0.07
2    Amer

Let's put that into a pandas dataframe

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Chinese Restaurant,Gas Station,Italian Restaurant,Pizza Place,Bubble Tea Shop,Sushi Restaurant,Fast Food Restaurant,Cosmetics Shop,Mobile Phone Shop
1,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Greek Restaurant,American Restaurant,Bar,Grocery Store,Sushi Restaurant,Thai Restaurant,Bagel Shop
2,Bedford Stuyvesant,Café,Deli / Bodega,Coffee Shop,Bar,Pizza Place,Park,Cocktail Bar,New American Restaurant,Thrift / Vintage Store,Bus Stop
3,Bensonhurst,Chinese Restaurant,Pizza Place,Bakery,Ice Cream Shop,Sushi Restaurant,Italian Restaurant,Grocery Store,Donut Shop,Dessert Shop,Liquor Store
4,Bergen Beach,Harbor / Marina,Donut Shop,Athletics & Sports,Other Repair Shop,Baseball Field,Hockey Field,Playground,Falafel Restaurant,Farm,Farmers Market


Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [64]:
# set number of clusters
kclusters = 5

Brooklyn_grouped_clustering = Brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 1, 3, 3, 1, 0, 3, 3, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [65]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Brooklyn_merged = Brooklyn_Data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Brooklyn_merged = Brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,3,Spa,Italian Restaurant,Pizza Place,Greek Restaurant,American Restaurant,Bar,Grocery Store,Sushi Restaurant,Thai Restaurant,Bagel Shop
1,Brooklyn,Bensonhurst,40.611009,-73.995180,3,Chinese Restaurant,Pizza Place,Bakery,Ice Cream Shop,Sushi Restaurant,Italian Restaurant,Grocery Store,Donut Shop,Dessert Shop,Liquor Store
2,Brooklyn,Sunset Park,40.645103,-74.010316,0,Bank,Bakery,Latin American Restaurant,Mexican Restaurant,Pizza Place,Mobile Phone Shop,Gym,Fried Chicken Joint,Pharmacy,Sandwich Place
3,Brooklyn,Greenpoint,40.730201,-73.954241,1,Bar,Pizza Place,Cocktail Bar,Coffee Shop,Yoga Studio,Sushi Restaurant,Grocery Store,French Restaurant,Record Shop,Spa
4,Brooklyn,Gravesend,40.595260,-73.973471,3,Italian Restaurant,Pizza Place,Bakery,Lounge,Chinese Restaurant,Bus Station,Music Store,Martial Arts Dojo,Furniture / Home Store,Music Venue


Finally, let's visualize the resulting clusters

In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Brooklyn_merged['Latitude'], Brooklyn_merged['Longitude'], Brooklyn_merged['Neighborhood'], Brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

#### Cluster 1(Commercial Places)

In [67]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 0, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Sunset Park,Bank,Bakery,Latin American Restaurant,Mexican Restaurant,Pizza Place,Mobile Phone Shop,Gym,Fried Chicken Joint,Pharmacy,Sandwich Place
10,East Flatbush,Print Shop,Hardware Store,Pharmacy,Park,Chinese Restaurant,Caribbean Restaurant,Moving Target,Food & Drink Shop,Supermarket,Department Store
25,Cypress Hills,Spanish Restaurant,Latin American Restaurant,Fried Chicken Joint,Donut Shop,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Ice Cream Shop,Video Store,Dance Studio
26,East New York,Deli / Bodega,Fried Chicken Joint,Caribbean Restaurant,Event Service,Fast Food Restaurant,Gym,Bus Stop,Metro Station,Bus Station,Pizza Place
27,Starrett City,Supermarket,Bus Station,Chinese Restaurant,Shopping Mall,Donut Shop,Pharmacy,Bus Stop,Caribbean Restaurant,Pizza Place,American Restaurant
28,Canarsie,Thai Restaurant,Chinese Restaurant,Gym,Caribbean Restaurant,Asian Restaurant,Women's Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
29,Flatlands,Pharmacy,Fried Chicken Joint,Caribbean Restaurant,Fast Food Restaurant,Park,Lounge,Discount Store,Bar,Video Store,Electronics Store
32,Coney Island,Baseball Stadium,Pizza Place,Caribbean Restaurant,Park,Brewery,Skating Rink,Music Venue,Beach,Theme Park Ride / Attraction,Food Court
34,Borough Park,Bank,Pizza Place,Fast Food Restaurant,Pharmacy,Metro Station,Deli / Bodega,American Restaurant,Hotel,Grocery Store,Restaurant
44,City Line,Donut Shop,Pharmacy,Pizza Place,Mobile Phone Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Lingerie Store,Liquor Store,Furniture / Home Store


#### Cluster 2(Tourist Areas & Hubs)

In [68]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 1, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Greenpoint,Bar,Pizza Place,Cocktail Bar,Coffee Shop,Yoga Studio,Sushi Restaurant,Grocery Store,French Restaurant,Record Shop,Spa
12,Windsor Terrace,Diner,Café,Plaza,Deli / Bodega,Park,Bagel Shop,Middle Eastern Restaurant,Chinese Restaurant,Bookstore,Sushi Restaurant
13,Prospect Heights,Bar,Mexican Restaurant,Café,Thai Restaurant,Cocktail Bar,Gourmet Shop,Bakery,Ice Cream Shop,Wine Shop,Coffee Shop
15,Williamsburg,Coffee Shop,Bar,Bagel Shop,Yoga Studio,Café,Taco Place,Korean Restaurant,Latin American Restaurant,Clothing Store,Steakhouse
16,Bushwick,Bar,Mexican Restaurant,Coffee Shop,Deli / Bodega,Discount Store,Thrift / Vintage Store,Pizza Place,Bakery,Vegetarian / Vegan Restaurant,Sandwich Place
17,Bedford Stuyvesant,Café,Deli / Bodega,Coffee Shop,Bar,Pizza Place,Park,Cocktail Bar,New American Restaurant,Thrift / Vintage Store,Bus Stop
18,Brooklyn Heights,Deli / Bodega,Yoga Studio,Park,Pizza Place,Mexican Restaurant,Italian Restaurant,Gym,Bakery,Cosmetics Shop,Diner
19,Cobble Hill,Playground,Pizza Place,Bar,Coffee Shop,Yoga Studio,Italian Restaurant,Ice Cream Shop,Cocktail Bar,Deli / Bodega,Middle Eastern Restaurant
20,Carroll Gardens,Italian Restaurant,Coffee Shop,Pizza Place,Bakery,Cocktail Bar,Wine Shop,Spa,Bar,Thai Restaurant,Food & Drink Shop
21,Red Hook,Seafood Restaurant,Art Gallery,Bar,American Restaurant,Park,Ice Cream Shop,New American Restaurant,Flower Shop,Pizza Place,Farm


#### Cluster 3(Fun Places)

In [69]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 2, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Pool,Women's Store,Filipino Restaurant,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop


#### Cluster 4(Center Acivity)

In [70]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 3, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Greek Restaurant,American Restaurant,Bar,Grocery Store,Sushi Restaurant,Thai Restaurant,Bagel Shop
1,Bensonhurst,Chinese Restaurant,Pizza Place,Bakery,Ice Cream Shop,Sushi Restaurant,Italian Restaurant,Grocery Store,Donut Shop,Dessert Shop,Liquor Store
4,Gravesend,Italian Restaurant,Pizza Place,Bakery,Lounge,Chinese Restaurant,Bus Station,Music Store,Martial Arts Dojo,Furniture / Home Store,Music Venue
5,Brighton Beach,Restaurant,Russian Restaurant,Eastern European Restaurant,Beach,Gourmet Shop,Sushi Restaurant,Bank,Mobile Phone Shop,Food & Drink Shop,Dessert Shop
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Sandwich Place,Fishing Spot,Martial Arts Dojo,Pizza Place,Buffet,Miscellaneous Shop,Fishing Store,Café
7,Manhattan Terrace,Pizza Place,Donut Shop,Ice Cream Shop,Bagel Shop,Burger Joint,Mobile Phone Shop,Steakhouse,Organic Grocery,Coffee Shop,Grocery Store
8,Flatbush,Chinese Restaurant,Deli / Bodega,Juice Bar,Coffee Shop,Mexican Restaurant,Caribbean Restaurant,Sandwich Place,Liquor Store,Lounge,Bank
9,Crown Heights,Pizza Place,Museum,Café,Bagel Shop,Moving Target,Playground,Candy Store,Supermarket,Liquor Store,Salon / Barbershop
11,Kensington,Thai Restaurant,Grocery Store,Pizza Place,Ice Cream Shop,Pharmacy,Mexican Restaurant,Sandwich Place,Mobile Phone Shop,Music Venue,Gas Station
14,Brownsville,Restaurant,Park,Spanish Restaurant,Fried Chicken Joint,Playground,Pizza Place,Pharmacy,Discount Store,Performing Arts Venue,Chinese Restaurant


#### Cluster 5 (Cultural & Going Out Places)

In [71]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 4, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Paerdegat Basin,Moving Target,Asian Restaurant,Gym,Food,Harbor / Marina,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field


## Results and Discussion <a name="results"></a>

After clustering the data of the respective neighborhoods, both cities (Boroughs) have venues which can be explored and attract the Tourists. The neighborhoods are much similar in features like Theaters, opera houses, food places, clubs, museums, parks etc. As far as concern to dissimilarity, it differs in terms of some unique places like historical places and monuments.
When we compare the tourist places, we observe that the park place is only situated in Central Toronto and the Monument or landmark venue is in Brooklyn neighborhoods. Similarly, Airport facility, Harbor, Sculpture garden and Boat or ferry services are also available in Central Toronto while venues like Nightlife, Climbing gym and Museums are present in Brooklyn.
As far as concern to recommendations, we recommend Brooklyn Neighborhoods will be considered first to visitfor more nightlife and fun.

## Conclusion <a name="conclusion"></a>

The Central Toronto and Brooklyn neighborhoods have more like similar venues. As we know that every place is unique in its own way, so that’s argument is present in both neighborhoods. The dissimilarity exists in terms of some different venues and facilities but not on a larger extent